In [8]:
import os
import pickle
import spacy
import pandas as pd
import numpy as np
from spacy.cli import download
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report

In [2]:
def fazer_amostragem(train_dataset: pd.DataFrame):
    """

    :param train_dataset:
    :return:
    """
    sentimentos = train_dataset['Sentimento'].unique()
    df = pd.DataFrame([])

    for sentimento in sentimentos:
        df_filtrado = train_dataset.loc[train_dataset['Sentimento'] == sentimento][:600]
        df = pd.concat([df, df_filtrado])

    return df


def ler_modelo(path: str):
    return pickle.load(open(path, 'rb'))


def salvar_modelo(path: str, modelo):
    return pickle.dump(modelo, open(path, 'wb'))    


def criar_embeddings(df_tweets):
    try:
        nlp = spacy.load('pt_core_news_lg')
    except (IOError, OSError):
        download('pt_core_news_lg')
        nlp = spacy.load('pt_core_news_lg')
    # desativamos todos os outros pipes que vem com o modelo nlp porque não preicsaremos deles
    with nlp.disable_pipes():
        # transformamos cada texto em um vetor e colocamos em uma array
        print('Fazendo os word embeddings')
        vetores = np.array([nlp(texto).vector for texto in df_tweets.Texto])

    return vetores


path_datasets = '../resources/datasets'

# importação dos dados
df_treinamento = pd.read_csv(
    f'{path_datasets}/tweets_ekman.csv',
    usecols=['Texto', 'Sentimento']
).dropna()
df_alvo = pd.read_csv(f'{path_datasets}/tweets_pandemia.csv')

classes = ["raiva", "triste"]
df_treinamento = df_treinamento[df_treinamento['Sentimento'].isin(classes)]

x_train, x_test, y_train, y_test = train_test_split(
    df_treinamento,
    df_treinamento['Sentimento'],
    test_size=0.2,
    random_state=42
)

In [10]:
# processamento dos dados para word embeddings
path_embeddings_treinamento = '../resources/modelos/embeddings_treinamento_logreg_2.pkl'
path_embeddings_teste = '../resources/modelos/embeddings_teste_logreg_2.pkl'

if os.path.exists(path_embeddings_treinamento):
    embeddings = ler_modelo(path_embeddings_treinamento)
else:
    embeddings = criar_embeddings(x_train)
    salvar_modelo(path_embeddings_treinamento, embeddings)

if os.path.exists(path_embeddings_teste):
    embeddings_teste = ler_modelo(path_embeddings_teste)
else:
    embeddings_teste = criar_embeddings(x_test)
    salvar_modelo(path_embeddings_teste, embeddings_teste)
# Train the first model
logreg_um = ler_modelo('../resources/modelos/logreg_otimizado.pkl')
logreg_um = logreg_um.best_estimator_
logreg_dois = LogisticRegression(random_state=0, n_jobs=-1)

In [11]:
logreg_dois.fit(embeddings, y_train)
previsoes_logreg_dois = logreg_dois.predict(embeddings_teste)
print(classification_report(y_test, previsoes_logreg_dois))

              precision    recall  f1-score   support

       raiva       0.66      0.28      0.40       551
      triste       0.84      0.96      0.90      2129

    accuracy                           0.82      2680
   macro avg       0.75      0.62      0.65      2680
weighted avg       0.80      0.82      0.79      2680


In [12]:
# importação dos dados
df_treinamento_final = pd.read_csv(
    f'{path_datasets}/tweets_ekman.csv',
    usecols=['Texto', 'Sentimento']
).dropna()
df_alvo_final = pd.read_csv(f'{path_datasets}/tweets_pandemia.csv')

x_train_final, x_test_final, y_train_final, y_test_final = train_test_split(
    df_treinamento_final,
    df_treinamento_final['Sentimento'],
    test_size=0.2,
    random_state=42
)

embeddings_final = ler_modelo('../resources/modelos/embeddings_treinamento.pkl')
embeddings_teste_final = ler_modelo('../resources/modelos/embeddings_teste.pkl')

In [19]:
# Create the voting classifier
classifier = VotingClassifier(estimators=[('lgr_um', logreg_um), ('lgr_dois', logreg_dois)], voting="hard", weights=[1, 2])
classifier.fit(embeddings_final, y_train_final)

# Make predictions
previsoes_final = classifier.predict(embeddings_teste_final)

# Evaluate the previsoes_final
print(classification_report(y_test_final, previsoes_final))

D:\home\Desenvolvimento\Analise-de-sentimentos-pandemia-covid19\emocoes_venv\Lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


              precision    recall  f1-score   support

      Neutro       0.97      0.94      0.96       495
       feliz       0.71      0.73      0.72      1931
        medo       0.54      0.61      0.57      2320
        nojo       0.61      0.63      0.62      1648
       raiva       0.45      0.12      0.19       562
      triste       0.52      0.53      0.53      2097

    accuracy                           0.61      9053
   macro avg       0.63      0.59      0.60      9053
weighted avg       0.60      0.61      0.60      9053


In [3]:
import pandas as pd

path_datasets = '../resources/datasets'
df = pd.read_csv(
    f'{path_datasets}/tweets_ekman.csv',
    usecols=['Texto', 'Sentimento']
).dropna()

df[df['Sentimento'].isin(['raiva'])]

,Texto,Sentimento
891,o que você acha que falta em sua vida?,raiva
1308,deixe ir o que não te deixa feliz.,raiva
4252,tudo o que você está com agora está impedindo...,raiva
10819,o que você acha que falta em sua vida?,raiva
10888,arrisque-se!,raiva
...,...,...
40161,transtorno explosivo intermitente: raiva sem l...,raiva
40408,agora vão ficar espalhando q ele passa fome aq...,raiva
41582,troca tudo!,raiva
41924,meu deus como esse povo de brasilia eh ridicul...,raiva
